In [1]:
#Imports
import sys
import os
import glob as glob
import pandas as pd
import numpy as np
from calvin_utils.file_utils.dataframe_utilities import preprocess_colnames_for_regression
np.set_printoptions(precision=3, suppress=True)

In [2]:
analysis = 'response_topology/voxelwise_mediated_moderation/age_mediated_by_grey_matter'
clin_path = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/study_metadata/derivative_metadata/quantitative_atrophy/grey_matter_damage_score_and_outcomes/grey_matter_damage_score_and_outcomes.csv'
out_dir = os.path.join(os.path.dirname(clin_path), f'{analysis}')
save = True
if os.path.exists(out_dir):
    pass
else:
    os.makedirs(out_dir)

Define Data to Submit

In [3]:
#----------------------------------------------------------------user input above----------------------------------------------------------------
data_df = pd.read_csv(clin_path)
    
# #Remove subjects
# outlier_index=[11, 47, 48, 49]
# data_df = data_df.drop(index=outlier_index)
# data_df.reset_index(drop=True, inplace=True)

data_df

,"Patient # CDR, ADAS",Age,% Change from baseline (ADAS-Cog11),Subiculum Connectivity,Subiculum Damage Score,Hippocampus Damage Score,Temporal Damage Score,Frontal Damage Score,Parietal Damage Score,Cerebellum Damage Score,Insula Damage Score,Occipital Damage Score
0,101,62,-21.428571,56.864683,-22.341183,-19032.589510,-6811.832831,-3390.513012,-6514.359274,-1733.125937,-133.189922,-4146.074230
1,102,77,-36.363636,52.970984,-40.309051,-184720.025100,-12864.565970,-6136.065565,-4548.582996,-5422.573735,-622.591915,-3965.251674
2,103,76,-78.947368,62.459631,-19.785287,-45387.958520,-6591.622671,-8075.228864,-4447.485410,-3607.025644,-624.924211,-2512.288860
3,104,65,-129.411765,59.611631,-42.332132,-60471.819480,-11699.796100,-6244.075797,-5212.034938,-3227.497127,-570.463181,-2934.581194
4,105,50,-10.526316,57.928350,-2.136441,-5968.011901,-4662.284230,-6429.216510,-4540.145985,-1893.423398,-299.251771,-2070.284124
5,106,66,-38.461538,63.323903,-26.237501,-29471.040290,-2479.701861,-2121.181555,-2841.442623,-1631.143552,-120.128438,-2222.047928
6,107,64,-15.384615,53.317851,-43.667344,-118201.273400,-11646.437390,-7046.077001,-8392.540599,-2361.515431,-614.793778,-6163.741802
7,108,60,-29.166667,57.979468,-62.535677,-66947.522780,-10926.588590,-4526.665072,-5782.261434,-3820.805562,-263.752339,-3537.170972
8,109,72,-30.434783,66.950749,-22.137900,-75270.490460,-11252.394020,-5240.470247,-4707.095249,-1171.115413,-261.459686,-3628.614588
9,110,72,-84.615385,69.267271,-6.545406,-40392.854920,-7111.854909,-9629.498968,-5124.972435,-1647.853988,-1035.648091,-2358.679568


Outocme Data

In [4]:
subject_column = 'Patient # CDR, ADAS'
outcome_column = '% Change from baseline (ADAS-Cog11)'
#----------------------------------------------------------------DO NOT MODIFY--------------------------------------------------------
outcomes_df = pd.DataFrame()
outcomes_df['outcome'] = data_df.loc[:, [outcome_column]]

#Standardize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
outcomes_df.iloc[:,:] = scaler.fit_transform(outcomes_df.iloc[:,:])


outcomes_df['subject_id'] = data_df.loc[:, [subject_column]]
# outcomes_df['subject_id'] = [id.split('_')[0] for id in data_df[subject_column].to_list()] #<------------------------problem code. must generalize to extract subject id better
outcomes_df.set_index('subject_id', inplace=True)

# data_df = data_df.set_index('Patient # CDR, ADAS')
# data_df['subject_id'] = data_df.index
# data_df['outcome'] = data_df

# Convert the 'subject_id' column to strings for each DataFrame
outcomes_df.index = outcomes_df.index.astype(str)

outcomes_df = preprocess_colnames_for_regression(outcomes_df)
display(outcomes_df)

,outcome
subject_id,
101,0.292279
102,-0.016545
103,-0.897080
104,-1.940569
105,0.517713
106,-0.059924
107,0.417254
108,0.132272
109,0.106051


Clinical Covariates

In [5]:
subject_column = 'Patient # CDR, ADAS'
clinical_information_column = 'Age'
#----------------------------------------------------------------DO NOT MODIFY--------------------------------------------------------
clinical_df_1 = pd.DataFrame()
clinical_df_1['clinical_data'] = data_df.loc[:, [clinical_information_column]]

clinical_df_1['subject_id'] = data_df.loc[:, [subject_column]]
# clinical_df_1['subject_id'] = [id.split('_')[0] for id in data_df[subject_column].to_list()] 
clinical_df_1.set_index('subject_id', inplace=True)

# Convert the 'subject_id' column to strings for each DataFrame
clinical_df_1.index = clinical_df_1.index.astype(str)
clinical_df_1 = preprocess_colnames_for_regression(clinical_df_1)
display(clinical_df_1)

,clinical_data
subject_id,
101,62
102,77
103,76
104,65
105,50
106,66
107,64
108,60
109,72


Voxelwise Data

In [6]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity'
matrix = import_matrices_from_folder(matrix_path, file_pattern='/*_T*.nii*')

# prepped_matrix = pd.DataFrame()
# for column in matrix.columns:
#     print(column)
#     new_name = int(column.split('_')[0]) #.split('T')[1]) #<----------------------------------THIS IS A BUG-CREATOR. MUST BE TAILORED TO PATIENT DATA

# #Set patients to those in the clinical data dataframe
matrix = matrix.transpose()
matrix['subject_id'] = [str(col).split('_')[0] for col in matrix.index]
matrix.set_index('subject_id', inplace=True)
matrix.index = matrix.index.astype(str)
neuroimaging_df_1 = preprocess_colnames_for_regression(matrix)
neuroimaging_df_1

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/*_T*.nii*
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_902619,var_902620,var_902621,var_902622,var_902623,var_902624,var_902625,var_902626,var_902627,var_902628
subject_id,,,,,,,,,,,,,,,,,,,,,
116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Voxelwise Data 2

In [7]:
#Import Another Dataframe if desired
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/cat12/cat12_ultrafine-reg/CAT12.8.2_2170'
neuroimaging_df_2 = import_matrices_from_folder(matrix_path, file_pattern='/*/*/*/*mwp1*resamp*.nii*')
performed_z_score = False 
#----------------------------------------------------------------DO NOT MODIFY!----------------------------------------------------------------

#Set patients to those in the clinical data dataframe
# prepped_matrix = prepped_matrix.loc[:, data_df.index]

neuroimaging_df_2 = neuroimaging_df_2.transpose()
neuroimaging_df_2['subject_id'] = [col.split('_')[0] for col in neuroimaging_df_2.index]
neuroimaging_df_2.set_index('subject_id', inplace=True)

neuroimaging_df_2.index = neuroimaging_df_2.index.astype(str)


#Display results
neuroimaging_df_2 = preprocess_colnames_for_regression(neuroimaging_df_2)
display(neuroimaging_df_2)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/cat12/cat12_ultrafine-reg/CAT12.8.2_2170/*/*/*/*mwp1*resamp*.nii*


,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_902619,var_902620,var_902621,var_902622,var_902623,var_902624,var_902625,var_902626,var_902627,var_902628
subject_id,,,,,,,,,,,,,,,,,,,,,
130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Masking

In [8]:
#Mask Dataframes
mask_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_glm/stim_by_age/palm_results/palm_statistic_topology/generated_nifti.nii' #None #
masking_df = None # # 
#----------------------------------------------------------------DO NOT MODIFY
from calvin_utils.nifti_utils.matrix_utilities import mask_matrix
# def mask_matrix(df_1, mask_path=None, mask_threshold=0.2, mask_by='rows', dataframe_to_mask_by=None):

neuroimaging_df_1 = mask_matrix(neuroimaging_df_1, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)
neuroimaging_df_2 = mask_matrix(neuroimaging_df_2, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)

Dataframes have been masked such that their shapes are:  (46, 37004)
Dataframes have been masked such that their shapes are:  (50, 37004)


Optional - Additional Preprocessing

In [9]:
neuroimaging_df_2 = neuroimaging_df_2.reindex(neuroimaging_df_1.index).dropna()
neuroimaging_df_2

,var_102956,var_102957,var_102958,var_102959,var_102960,var_103046,var_103047,var_103048,var_103049,var_103050,...,var_797919,var_798009,var_798010,var_798099,var_798100,var_798190,var_798191,var_798281,var_798282,var_798373
subject_id,,,,,,,,,,,,,,,,,,,,,
116,0.229362,0.205727,0.053395,0.074691,0.113833,0.423250,0.513379,0.377440,0.153252,0.007786,...,0.463366,0.303836,0.372540,0.605460,0.464765,0.746839,0.739713,0.870503,0.752853,0.769740
139,0.109401,0.150143,0.197897,0.425160,0.362449,0.138988,0.176965,0.240504,0.407940,0.543465,...,0.066427,0.405873,0.192811,0.508609,0.499546,0.555987,0.478337,0.583788,0.478562,0.429630
147,0.162510,0.179246,0.263863,0.298417,0.158954,0.351168,0.314347,0.409709,0.518645,0.382331,...,0.110859,0.403632,0.583603,0.690095,0.910620,0.837578,0.951557,0.822104,0.886667,0.739561
135,0.153985,0.113712,0.141172,0.173383,0.114053,0.193699,0.248250,0.240887,0.267419,0.182920,...,0.068370,0.154941,0.097959,0.284367,0.265228,0.388628,0.386707,0.442965,0.473160,0.499106
101,0.364223,0.145639,0.089289,0.140574,0.196354,0.665556,0.729897,0.594886,0.243001,0.039168,...,0.227021,0.293693,0.213777,0.688021,0.515107,0.662774,0.660089,0.682468,0.690582,0.717662
150,0.246922,0.405237,0.310447,0.300769,0.206188,0.243030,0.413811,0.660632,0.502521,0.380426,...,0.058324,0.354329,0.274899,0.790549,0.449493,0.734528,0.539761,0.682080,0.574928,0.593957
122,0.204949,0.197069,0.196635,0.160092,0.044966,0.266371,0.292215,0.281765,0.244403,0.121008,...,0.118555,0.191085,0.216092,0.368782,0.331164,0.385139,0.412472,0.474175,0.450113,0.474031
146,0.175004,0.321799,0.394048,0.364054,0.238638,0.406729,0.353518,0.489140,0.512552,0.508472,...,0.404712,0.447947,0.505782,0.575915,0.614407,0.668394,0.675969,0.663611,0.682129,0.510989
138,0.188685,0.139497,0.121198,0.156062,0.143464,0.292154,0.373723,0.256809,0.156228,0.161139,...,0.211858,0.634101,0.550631,0.687401,0.573499,0.555129,0.571267,0.448494,0.551988,0.422085


# Save Dataframes for Access by Script
    """
    Saves DataFrames to CSV files and returns the paths.

    Parameters:
    - outcome_dfs (list): A list of outcome DataFrames.
    - covariate_dfs (list): A list of covariate DataFrames.
    - voxelwise_dfs (list): A list of voxelwise DataFrames.
    - path_to_dataframes (str): The directory where the DataFrames will be saved.

    Returns:
    - dict: A dictionary containing lists of paths for each DataFrame type.
    """

In [10]:
from calvin_utils.file_utils.dataframe_utilities import save_dataframes_to_csv

df_paths_dict = save_dataframes_to_csv(outcome_dfs = [outcomes_df], 
                                       covariate_dfs = [clinical_df_1],
                                       voxelwise_dfs = [neuroimaging_df_1], 
                                       path_to_dataframes = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/f_test/server_prep')
print(df_paths_dict)

{'outcomes': ['/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/f_test/server_prep/outcome_data_1.csv'], 'covariates': ['/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/f_test/server_prep/covariate_data_1.csv'], 'voxelwise': ['/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/f_test/server_prep/voxelwise_data_1.csv']}


# Submit to cluster


**Display Statistical Methods You Can Use**

In [11]:
from calvin_utils.file_utils.script_printer import ScriptInfo
from calvin_utils.permutation_analysis_utils.scripts_for_submission.script_descriptions import script_dict
#----------------------------------------------------------------
info = ScriptInfo(script_dict)
info.print_high_level_info()

Script: launch_f_test_palm.py
Method: F-Test
---


**Choose Method**

Copy the method you want to run fromt the output above into the cell below


In [12]:
method_to_use = 'F-Test'

**Transfer Files To Server**

This is going to upload the data and the script you will use.
___

    ScpTransfer class provides a convenient way to transfer files to a remote server
    using SCP (Secure Copy).
    Will request your password for transfer if no ssh_key provided.

    Example:
        remote_base_path = "/path/on/remote/server"
        remote_hostname = "example.com"
        remote_username = "your_username"
        ssh_key = "path/to/ssh_key/"

        scp_transfer = ScpTransfer(remote_hostname, remote_username, remote_password)
        scp_transfer.transfer_files_in_dict(dict_files_to_transfer, remote_base_path)

In [13]:
import importlib
from calvin_utils.server_utils.file_transfer_helper import ScpTransfer

#----------------------------------------------------------------
df_paths_dict['python_script'] = [importlib.import_module(info.get_module_by_method(method_to_use)).__file__]
scp_transfer = ScpTransfer(hostname="erisone.partners.org",
                           username="cu135", 
                           ssh_key=None
                        )
remote_df_paths_dict = scp_transfer.transfer_files_in_dict(dict_files=df_paths_dict, 
                                    base_remote_path='/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs')

Successfully transferred /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/f_test/server_prep/outcome_data_1.csv to /PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/outcomes/outcome_data_1.csv.
Successfully transferred /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/f_test/server_prep/covariate_data_1.csv to /PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/covariates/covariate_data_1.csv.
Successfully transferred /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/f_test/server_prep/voxelwise_data_1.csv to /PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/voxelwise/voxelwise_data_1.csv.
Successfully transferred /Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/calvin_utils/calvin_utils/permutation_analysis_utils/scripts_f

**Print Input Arguments and Dictionary for Script**

In [14]:
#----------------------------------------------------------------
inputs = info.get_inputs_by_method(method_to_use)
inputs

{'n_cores': 'The number of cores per job submission cpu (4 is a good default).',
 'out_dir': 'The output directory where the result csv files will be saved.',
 'job_name': 'The job name for identification.',
 'memory_per_job': 'The memory (in gigabytes) per job submission cpu. General max is 40 up to 498 Gb.',
 'outcome_data_path': 'The path to the outcome data csv file.',
 'clinical_covariate_paths': 'The paths to the clinical covariate data csv files.',
 'neuroimaging_df_paths': 'The paths to the voxelwise neuroimaging data csv files.'}

**Enter Arguments for a Script**

Copy the dictionary printed above into the cell below and fill it out. 

For example
 
 script_inputs_dict =
 
  {
    
 'n_cores': 16,

 'out_dir': "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/results/tmp",

 'job_name': 'ftest_bm',

 'outcome_data_path': remote_df_paths_dict['outcomes'],

 'clinical_covariate_paths': remote_df_paths_dict['covariates'],

 'voxelwise_data_paths': remote_df_paths_dict['voxelwise']
 
 }

In [15]:

script_inputs_dict = {
 'n_cores': 5,
 'out_dir': "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/results/big_tmp",
 'job_name': 'ftest_normal',
 'memory_per_job': 8,
 'outcome_data_path': remote_df_paths_dict['outcomes'],
 'clinical_covariate_paths': remote_df_paths_dict['covariates'],
 'neuroimaging_df_paths': remote_df_paths_dict['voxelwise']
 }

**Submit Jobs**

    Initialize server
    server = LSFServer("erisone.partners.org", "cu135")

    Initialize job
    job = LSFJob(
    job_name="f_test_bm",
    user_email="choward12@bwh.harvard.edu",
    cpus=1,
    output_dir="/PHShome/cu135/terminal_outputs",
    error_dir="/PHShome/cu135/error_outputs",
    queue="big-multi",
    script_path="/PHShome/cu135/python_scripts/launch_f_test_palm.py",
    options="-o /PHShome/cu135/permutation_tests/f_test/age_by_stim_pd_dbs/results",
    Gb_requested=4,
    wait_time=0
    )

    Initialize job submitter
    job_submitter = JobSubmitter(server, job)

    Submit jobs
    job_submitter.submit_jobs(10)

This example will log into the server 'erisone.partners.org' as user 'cu135'.
 
Will submit 10 jobs named 'f_test_bm' to the queue 'big-multi'. 

Each job will run the Python script at '/PHShome/cu135/python_scripts/launch_f_test_palm.py' with the options specified.

Putputting to '/PHShome/cu135/terminal_outputs' and '/PHShome/cu135/error_outputs' for terminal and error logs respectively. 

The job requests 1 CPU and 4GB of memory.


For a guide to job submission:

https://rc.partners.org/kb/article/1462

In [16]:
from calvin_utils.server_utils.job_submission_helper import LSFServer, LSFJob, JobSubmitter
# Create an instance of the LSF job.
# Assuming a resource requirement of 500GB memory and 6 cores for each job.
# And a job script with some options.
server_name = "erisone.partners.org"
user_email = "choward12@bwh.harvard.edu"
user_name = "cu135"
num_permutations = 10
queue_name = "normal"
server_env_activation_string = None # "conda activate nimlab"
#----------------------------------------------------------------
lsf_job = LSFJob(job_name=script_inputs_dict['job_name'],
                 user_email=user_email,
                 output_dir=script_inputs_dict['out_dir'],
                 error_dir=f"~/terminal_outputs",
                 queue=queue_name,
                 n_jobs=int(np.round(num_permutations/script_inputs_dict['n_cores'])),
                 cpus=script_inputs_dict['n_cores'],
                 gb_requested=script_inputs_dict['memory_per_job'],
                 wait_time=None,
                 script_path=remote_df_paths_dict['python_script'][0],
                 environment_activation_string=server_env_activation_string,
                 options= " ".join(f'--{key} "{value}"' for key, value in script_inputs_dict.items())
                 )

lsf_server = LSFServer(server_name=server_name, 
                       username=user_name)

job_submitter = JobSubmitter(lsf_server, lsf_job)
job_submitter.submit_jobs(print_job=True)


Job command:  bsub -q normal -n 5 -R 'rusage[mem=8000] span[ptile=5]' -M 8000 -o /PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/results/big_tmp/ftest_normal_%I.txt -J 'ftest_normal[1-2]' -u choward12@bwh.harvard.edu -B -N -cwd /PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/python_script && python /PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/python_script/launch_f_test_palm.py --n_cores "5" --out_dir "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/results/big_tmp" --job_name "ftest_normal" --memory_per_job "8" --outcome_data_path "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/outcomes/outcome_data_1.csv" --clinical_covariate_paths "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/covariates/covariate_data_1.csv" --neuroimaging_df_paths "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/voxelwise/voxelwise_data_1.csv"'
Job s

In [ ]:
bsub -q big -n 5 -R 'rusage[mem=8000] span[ptile=5]' -M 8000 -o /PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/results/big_tmp/ftest_normal_%I.txt -J 'ftest_normal[1-2]' -u choward12@bwh.harvard.edu -B -N -cwd /PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/python_script python /PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/python_script/launch_f_test_palm.py --n_cores "5" --out_dir "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/results/big_tmp" --job_name "ftest_normal" --memory_per_job "8" --outcome_data_path "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/outcomes/outcome_data_1.csv" --clinical_covariate_paths "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/covariates/covariate_data_1.csv" --neuroimaging_df_paths "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/voxelwise/voxelwise_data_1.csv"











In [18]:
bsub -q big -n 5 -R 'rusage[mem=8000] span[ptile=5]' -M 8000 -o /PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/results/big_tmp/ftest_normal_%I.txt -J 'ftest_normal[1-2]' -u choward12@bwh.harvard.edu -B -N -cwd /PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/python_script python /PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/python_script/launch_f_test_palm.py --n_cores "5" --out_dir "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/results/big_tmp" --job_name "ftest_normal" --memory_per_job "8" --outcome_data_path "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/outcomes/outcome_data_1.csv" --clinical_covariate_paths "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/covariates/covariate_data_1.csv" --neuroimaging_df_paths "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/inputs/voxelwise/voxelwise_data_1.csv"


SyntaxError: invalid syntax (963598015.py, line 1)

# Recompose Data
This code uses a file-staging approach to large-scale computation. The resultant files have been saved to your output directory. You must now recompose them. 

In [ ]:

import os
import pandas as pd

def combine_csvs(directory, output_filename):
    """
    Combine all CSV files in a directory into a single CSV file.

    Parameters:
    - directory (str): The path to the directory containing the CSV files.
    - output_filename (str): The path to the output CSV file.
    """
    # Initialize an empty DataFrame
    combined_df = pd.DataFrame()

    # Get a list of all CSV files in the directory
    csv_files = [f for f in os.listdir(directory) if f.endswith(".csv")]

    # Loop through the CSV files and append each one to the combined DataFrame
    for csv_file in csv_files:
        df = pd.read_csv(os.path.join(directory, csv_file))
        combined_df = pd.concat([combined_df, df])

    # Save the combined DataFrame as a new CSV file
    combined_df.to_csv(output_filename, index=False)
    return output_filename


In [ ]:
recomposed_csv_path = combine_csvs("/path/to/your/directory", "combined.csv")
recomposed_csv_df = pd.read_csv(recomposed_csv_path)
recomposed_csv_df

In [ ]:
#import confidence_intervals
# run confidence intervals on the csv

import multiprocessing as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
from statsmodels.formula.api import ols

def calculate_confidence_intervals(ab_paths, mediators):
    """
    Calculates the confidence intervals and p-value based on the bootstrapped samples.

    Parameters:
    - ab_paths: list of lists containing the bootstrapped ab paths for each mediator.
    - total_indirect_effects: list of bootstrapped summed ab paths.
    - mediators: list of mediator names.

    Returns:
    - DataFrame with the mean indirect effect, confidence intervals, and p-values for each mediator and the total indirect effect.
    """
    ab_path_values = np.array(ab_paths)

    # Check if there's only one mediator
    if isinstance(mediators, str):
        mediators = [mediators]

    # Calculate mean indirect effect and confidence intervals for each mediator
    mean_ab_paths = np.mean(ab_path_values, axis=0)
    lower_bounds = np.percentile(ab_path_values, 2.5, axis=0)
    upper_bounds = np.percentile(ab_path_values, 97.5, axis=0)

    # Calculate p-values for each mediator
    ab_path_p_values = [np.mean(np.sign(mean_ab_paths) * ab_path_values <= 0)]

    # Create DataFrame to store the results
    result_df = pd.DataFrame({
        'Point Estimate': mean_ab_paths,
        '2.5th Percentile': lower_bounds,
        '97.5th Percentile': upper_bounds,
        'P-value': ab_path_p_values
    }, index=mediators)

    return result_df


Get Results

In [ ]:
pool = mp.Pool(mp.cpu_count())
results = list(tqdm(pool.imap(calculate_confidence_intervals, [row for _, row in recomposed_csv_df.iterrows()]), total=recomposed_csv_df.shape[0]))
pool.close() 

# Create the results DataFrame
results_df = pd.concat(results, axis=1).transpose()

# Create Nifti Results

In [ ]:
# results_df = pd.read_csv('/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/mediated_moderation/age_mediated_by_grey_matter/P-value_mediated_moderation_analysis_results.csv')

mask_threshold=0
#----------------------------------------------------------------DO NOT TOUCH
#Prepare Indices to Unmask
from calvin_utils.nifti_utils.matrix_utilities import unmask_matrix
from nimlab import datasets as nimds
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
from nilearn import image

#Perform Unmasking
if mask_path is not None:
    mni_mask = image.load_img(mask_path).get_fdata().flatten()
    brain_indices = np.where(mni_mask > mask_threshold)[0]
elif masking_df is not None:
    mni_mask = nimds.get_img("mni_icbm152").get_fdata().flatten()
    mask = masking_df.transpose().reset_index(drop=True).copy()
    mask['mask_index'] = mask.sum(axis=1)
    brain_indices = np.where(mask['mask_index'] != 0)[0]
else:
    mni_mask = nimds.get_img("mni_icbm152").get_fdata().flatten()
    brain_indices = np.where(mni_mask > 0)[0]

# Create a boolean mask for brain_indices
bool_mask = np.zeros_like(mni_mask, dtype=bool)
bool_mask[brain_indices] = True

for statistic in results_df.columns:
    try:
        print(statistic)
        
        # Initialize the output mask with NaN values
        output_mask = np.full_like(mni_mask, np.nan)
        
        # Reinstate the values at brain_indices
        output_mask[bool_mask] = results_df[statistic]
        
        # View and save
        view_and_save_nifti(output_mask, out_dir=out_dir, output_name=statistic)
    except:
        print(f"Couldn't convert {statistic} to nifti")

Done